<a href="https://colab.research.google.com/github/Shravyap08/NLP-projects/blob/main/Disaster_management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#IMPORTING LIBRARIES

import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score


In [2]:
!pip install torch

ERROR: Operation cancelled by user


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#LOADING DATA

df_train = pd.read_csv("/content/drive/MyDrive/Disaster classification LLM/Data/train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Disaster classification LLM/Data/test.csv")
df_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [5]:
#SPLITTING DATA
X_train, X_test, y_train, y_test = train_test_split(df_train['text'], df_train['target'], test_size=0.2)



In [6]:
#TOKENIZING DATA

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples, padding='max_length',truncation=True, max_length=128)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
#Converting X(text) to encodings

train_encodings = tokenize_function(X_train.tolist())
val_encodings= tokenize_function(X_test.tolist())
test_encodings = tokenize_function(df_test['text'].tolist())



In [8]:
#Converting y(labels) to encodings
train_labels= torch.tensor(y_train.values)
val_labels= torch.tensor(y_test.values)


In [9]:
# CUSTOMIZE DATASET
class TweetDataset(Dataset):
    def __init__(self, encodings,labels):
        self.encodings= encodings
        self.labels=labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item


    def __len__(self):
        return len(self.labels)

In [10]:
train_dataset = TweetDataset(train_encodings, train_labels)
val_dataset = TweetDataset(val_encodings, val_labels)

In [11]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [12]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [14]:
# Training loop
from torch.nn.functional import softmax
model.train()
for epoch in range(3):  # Number of epochs
    total_loss = 0
    all_preds = []
    all_labels = []

    for batch_idx, batch in enumerate(train_loader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        probs = softmax(outputs.logits, dim=1)
        preds = torch.argmax(probs, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())



        # Optionally, print the loss every certain number of batches
        if (batch_idx + 1) % 10 == 0:  # Adjust this number based on your dataset size and batch size
            print(f'Epoch: {epoch+1}, Batch: {batch_idx+1}, Loss: {loss.item():.4f}')



    # Print average loss per epoch
    average_loss = total_loss / len(train_loader)
    print(f'End of Epoch {epoch+1}, Average Loss: {average_loss:.4f}')

    f1 = f1_score(all_labels, all_preds, average='weighted')
    print(f'End of Epoch {epoch+1}, F1 Score: {f1:.4f}')

Epoch: 1, Batch: 10, Loss: 0.6980
Epoch: 1, Batch: 20, Loss: 0.5808
Epoch: 1, Batch: 30, Loss: 0.7118
Epoch: 1, Batch: 40, Loss: 0.5817
Epoch: 1, Batch: 50, Loss: 0.4170
Epoch: 1, Batch: 60, Loss: 0.4466
Epoch: 1, Batch: 70, Loss: 0.5500
Epoch: 1, Batch: 80, Loss: 0.5092
Epoch: 1, Batch: 90, Loss: 0.4124
Epoch: 1, Batch: 100, Loss: 0.2911
Epoch: 1, Batch: 110, Loss: 0.4783
Epoch: 1, Batch: 120, Loss: 0.3566
Epoch: 1, Batch: 130, Loss: 0.3194
Epoch: 1, Batch: 140, Loss: 0.4197
Epoch: 1, Batch: 150, Loss: 0.3177
Epoch: 1, Batch: 160, Loss: 0.3022
Epoch: 1, Batch: 170, Loss: 0.3146
Epoch: 1, Batch: 180, Loss: 0.4112
Epoch: 1, Batch: 190, Loss: 0.4821
Epoch: 1, Batch: 200, Loss: 0.4937
Epoch: 1, Batch: 210, Loss: 0.3266
Epoch: 1, Batch: 220, Loss: 0.2835
Epoch: 1, Batch: 230, Loss: 0.3778
Epoch: 1, Batch: 240, Loss: 0.3566
Epoch: 1, Batch: 250, Loss: 0.4159
Epoch: 1, Batch: 260, Loss: 0.4386
Epoch: 1, Batch: 270, Loss: 0.2703
Epoch: 1, Batch: 280, Loss: 0.4008
Epoch: 1, Batch: 290, Loss: 0

In [17]:
#Testing/ Validating loop

model.eval()
for epoch in range(1):  # Number of epochs
    total_loss = 0
    all_preds = []
    all_labels = []

    for batch_idx, batch in enumerate(val_loader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        probs = softmax(outputs.logits, dim=1)
        preds = torch.argmax(probs, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())



        # Optionally, print the loss every certain number of batches
        if (batch_idx + 1) % 10 == 0:  # Adjust this number based on your dataset size and batch size
            print(f'Epoch: {epoch+1}, Batch: {batch_idx+1}, Loss: {loss.item():.4f}')



    # Print average loss per epoch
    average_loss = total_loss / len(val_loader)
    print(f'End of Epoch {epoch+1}, Average Loss: {average_loss:.4f}')

    f1 = f1_score(all_labels, all_preds, average='weighted')
    print(f'End of Epoch {epoch+1}, F1 Score: {f1:.4f}')

Epoch: 1, Batch: 10, Loss: 0.0161
Epoch: 1, Batch: 20, Loss: 0.0516
Epoch: 1, Batch: 30, Loss: 0.0295
Epoch: 1, Batch: 40, Loss: 0.0243
Epoch: 1, Batch: 50, Loss: 0.0373
Epoch: 1, Batch: 60, Loss: 0.0143
Epoch: 1, Batch: 70, Loss: 0.0557
Epoch: 1, Batch: 80, Loss: 0.0062
Epoch: 1, Batch: 90, Loss: 0.0493
End of Epoch 1, Average Loss: 0.0642
End of Epoch 1, F1 Score: 0.9789
